In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
%matplotlib inline
from tensorflow.keras.preprocessing import image
import random
import pandas as pd

In [ ]:
import pickle
pickle_file_path = 'labeled_data.pkl'

# Load the pickle file
with open(pickle_file_path, 'rb') as f:
    labeled_data = pickle.load(f)

In [ ]:
import pickle
pickle_file_path = 'filtered_df.pkl'

# Load the pickle file
with open(pickle_file_path, 'rb') as f:
    filtered_df = pickle.load(f)

In [ ]:
filtered_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg


In [ ]:
import random

ind = filtered_df.index.tolist()
random.shuffle(ind)

In [ ]:
n = len(filtered_df)
p_train = 0.6
p_val = 0.2
n_train = int(p_train*n)
n_val = int(p_val*n)
train_ind = ind[:n_train]
val_ind = ind[n_train:(n_train+n_val)]
test_ind = ind[(n_train+n_val):]

In [ ]:
train_img = []
val_img = []
test_img = []
train_label = []
val_label = []
test_label = []
test_ids = []

for img in labeled_data:
    if img['index'] in train_ind:
        train_img.append(img['img'])
        train_label.append(img['label'])
    elif img['index'] in val_ind:
        val_img.append(img['img'])
        val_label.append(img['label'])
    elif img['index'] in test_ind:
        test_img.append(img['img'])
        test_label.append(img['label'])
        test_ids.append(img['index'])

In [ ]:
from datasets import Dataset

train_ds = Dataset.from_dict({'img':train_img,'label':train_label})
val_ds = Dataset.from_dict({'img':val_img,'label':val_label})
test_ds = Dataset.from_dict({'img':test_img,'label':test_label})

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
from datasets import Dataset

# 1. Load the Pretrained ResNet Model
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetFeatureExtractor, self).__init__()
        resnet = models.resnet50(pretrained=pretrained)
        # Remove the final classification layer (fc)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Keep all layers except the last FC layer

    def forward(self, x):
        features = self.feature_extractor(x)
        return features.view(features.size(0), -1)  # Flatten the features

In [ ]:
# Option 2: Using Hugging Face AutoFeatureExtractor
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")

# You can then use either one in your preprocessing function:
def preprocess_images(examples):
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays
    images = [np.array(image, dtype=np.uint8) for image in images]
    
    if isinstance(feature_extractor, transforms.Compose):
        # For torchvision transforms
        images = [Image.fromarray(img) for img in images]
        images = [feature_extractor(img) for img in images]
        examples['pixel_values'] = torch.stack(images)
    else:
        # For Hugging Face feature extractor
        inputs = feature_extractor(images=images, return_tensors="pt")
        examples['pixel_values'] = inputs['pixel_values']
    
    return examples

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

c:\Users\ASUS\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--microsoft--resnet-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The c

In [ ]:

top_labels = pd.DataFrame(filtered_df.groupby('articleType').size().reset_index().sort_values(0,ascending = False)[:11]['articleType'])
top_labels_list = sorted(list(top_labels['articleType']))
top_labels['label_num'] = top_labels['articleType'].apply(lambda x: top_labels_list.index(x))
top_labels

,articleType,label_num
7,Tshirts,7
4,Shirts,4
0,Casual Shoes,0
9,Watches,9
5,Sports Shoes,5
3,Kurtas,3
6,Tops,6
1,Handbags,1
2,Heels,2
8,Wallets,8


In [ ]:
def preprocess_images(examples):
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays
    images = [np.array(image, dtype=np.uint8) for image in images]
    
    if isinstance(feature_extractor, transforms.Compose):
        # For torchvision transforms
        images = [Image.fromarray(img) for img in images]
        processed_images = [feature_extractor(img) for img in images]
        # Convert tensor to numpy array for storage
        processed_images = [img.numpy() for img in processed_images]
    else:
        # For Hugging Face feature extractor
        inputs = feature_extractor(images=images, return_tensors="pt")
        # Convert tensor to numpy array for storage
        processed_images = inputs['pixel_values'].numpy()
    
    examples['pixel_values'] = processed_images
    return examples

# Define features
features = Features({
    'label': ClassLabel(names=top_labels_list),
    'img': Array3D(dtype="int64", shape=(3, 32, 32)),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224))
})

# Process datasets
preprocessed_train_ds = train_ds.map(
    preprocess_images, 
    batched=True, 
    features=features,
    batch_size=32
)
preprocessed_val_ds = val_ds.map(
    preprocess_images, 
    batched=True, 
    features=features,
    batch_size=32
)
preprocessed_test_ds = test_ds.map(
    preprocess_images, 
    batched=True, 
    features=features,
    batch_size=32
)

Map:   0%|          | 0/1612 [00:00<?, ? examples/s]

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Map:   0%|          | 0/524 [00:00<?, ? examples/s]

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn

class CustomResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet, self).__init__()
        # Load pretrained ResNet
        self.resnet = models.resnet50(pretrained=True)
        # Replace the final fully connected layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
        
    def forward(self, pixel_values, labels=None):
        # Get the logits from ResNet
        logits = self.resnet(pixel_values)
        
        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=None,
            attentions=None,
        )

# Initialize the model
model = CustomResNet(num_classes=len(top_labels_list))

metric_name = "accuracy"

# Training arguments remain the same
args = TrainingArguments(
    f"test-clothing",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
)

c:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    accuracy = (preds == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

# 7. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_train_ds,
    eval_dataset=preprocessed_val_ds,
    tokenizer=None,  # No tokenizer required for vision models
    compute_metrics=compute_metrics,
)

# 8. Train the Model
trainer.train()



C:\Users\ASUS\AppData\Local\Temp\ipykernel_15912\3491551552.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/2828 [00:00<?, ?it/s]

{'loss': 2.43, 'grad_norm': 18.632522583007812, 'learning_rate': 1.9929278642149933e-05, 'epoch': 0.05}
{'loss': 2.068, 'grad_norm': 13.595705032348633, 'learning_rate': 1.985855728429986e-05, 'epoch': 0.1}
{'loss': 1.8515, 'grad_norm': 17.159460067749023, 'learning_rate': 1.978783592644979e-05, 'epoch': 0.15}
{'loss': 1.5488, 'grad_norm': 16.069259643554688, 'learning_rate': 1.971711456859972e-05, 'epoch': 0.2}
{'loss': 1.3936, 'grad_norm': 11.296889305114746, 'learning_rate': 1.9646393210749647e-05, 'epoch': 0.25}
{'loss': 1.3092, 'grad_norm': 11.984753608703613, 'learning_rate': 1.957567185289958e-05, 'epoch': 0.3}
{'loss': 1.1375, 'grad_norm': 12.58011245727539, 'learning_rate': 1.9504950495049508e-05, 'epoch': 0.35}
{'loss': 1.1929, 'grad_norm': 10.247995376586914, 'learning_rate': 1.9434229137199436e-05, 'epoch': 0.4}
{'loss': 1.1302, 'grad_norm': 13.451748847961426, 'learning_rate': 1.9363507779349364e-05, 'epoch': 0.45}
{'loss': 1.0856, 'grad_norm': 11.160205841064453, 'learnin

  0%|          | 0/68 [00:00<?, ?it/s]

{'eval_loss': 0.5873302817344666, 'eval_accuracy': 0.7759259343147278, 'eval_runtime': 32.7972, 'eval_samples_per_second': 16.465, 'eval_steps_per_second': 2.073, 'epoch': 1.0}
{'loss': 0.6416, 'grad_norm': 11.441851615905762, 'learning_rate': 1.8514851485148516e-05, 'epoch': 1.04}
{'loss': 0.4801, 'grad_norm': 7.6913862228393555, 'learning_rate': 1.8444130127298444e-05, 'epoch': 1.09}
{'loss': 0.68, 'grad_norm': 13.041190147399902, 'learning_rate': 1.8373408769448376e-05, 'epoch': 1.14}
{'loss': 0.5587, 'grad_norm': 13.647153854370117, 'learning_rate': 1.8302687411598305e-05, 'epoch': 1.19}
{'loss': 0.6204, 'grad_norm': 12.076216697692871, 'learning_rate': 1.8231966053748233e-05, 'epoch': 1.24}
{'loss': 0.5768, 'grad_norm': 10.498698234558105, 'learning_rate': 1.8161244695898165e-05, 'epoch': 1.29}
{'loss': 0.6335, 'grad_norm': 4.55781364440918, 'learning_rate': 1.809052333804809e-05, 'epoch': 1.34}
{'loss': 0.7004, 'grad_norm': 14.792625427246094, 'learning_rate': 1.8019801980198022e

...

  0%|          | 0/68 [00:00<?, ?it/s]

{'eval_loss': 0.3516092002391815, 'eval_accuracy': 0.8888888955116272, 'eval_runtime': 33.8246, 'eval_samples_per_second': 15.965, 'eval_steps_per_second': 2.01, 'epoch': 3.0}
{'loss': 0.2367, 'grad_norm': 15.738625526428223, 'learning_rate': 1.5685997171145687e-05, 'epoch': 3.02}
{'loss': 0.1318, 'grad_norm': 8.589910507202148, 'learning_rate': 1.5615275813295616e-05, 'epoch': 3.07}
{'loss': 0.1106, 'grad_norm': 17.403480529785156, 'learning_rate': 1.5544554455445548e-05, 'epoch': 3.12}
{'loss': 0.1734, 'grad_norm': 7.151615619659424, 'learning_rate': 1.5473833097595473e-05, 'epoch': 3.17}
{'loss': 0.1643, 'grad_norm': 5.025076389312744, 'learning_rate': 1.5403111739745405e-05, 'epoch': 3.22}
{'loss': 0.2189, 'grad_norm': 9.948522567749023, 'learning_rate': 1.5332390381895333e-05, 'epoch': 3.27}
{'loss': 0.156, 'grad_norm': 3.644188642501831, 'learning_rate': 1.526166902404526e-05, 'epoch': 3.32}
{'loss': 0.1731, 'grad_norm': 9.135891914367676, 'learning_rate': 1.5190947666195193e-05,

  0%|          | 0/68 [00:00<?, ?it/s]

{'eval_loss': 0.3577110171318054, 'eval_accuracy': 0.8833333253860474, 'eval_runtime': 32.9939, 'eval_samples_per_second': 16.367, 'eval_steps_per_second': 2.061, 'epoch': 4.0}
{'loss': 0.2026, 'grad_norm': 2.9583137035369873, 'learning_rate': 1.4271570014144273e-05, 'epoch': 4.01}


KeyboardInterrupt: 

In [ ]:
# 9. Evaluate the Model on Test Set
test_results = trainer.evaluate(preprocessed_test_ds)
print(f"Test Results: {test_results}")

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.4226503372192383, 'eval_accuracy': 0.8625954389572144, 'eval_runtime': 25.0073, 'eval_samples_per_second': 20.954, 'eval_steps_per_second': 2.639, 'epoch': 4.04}
Test Results: {'eval_loss': 0.4226503372192383, 'eval_accuracy': 0.8625954389572144, 'eval_runtime': 25.0073, 'eval_samples_per_second': 20.954, 'eval_steps_per_second': 2.639, 'epoch': 4.044554455445544}


In [ ]:
outputs = trainer.predict(preprocessed_test_ds)

  0%|          | 0/66 [00:00<?, ?it/s]

In [ ]:
print(outputs.metrics)

{'test_loss': 0.4226503372192383, 'test_accuracy': 0.8625954389572144, 'test_runtime': 22.758, 'test_samples_per_second': 23.025, 'test_steps_per_second': 2.9}


In [ ]:
# Save the model's state dictionary
torch.save(model.state_dict(), r"C:\Users\ASUS\Desktop\shoppin-assignment\resnet-model-trained\model.pth")

# Save the feature extractor
feature_extractor.save_pretrained(r"C:\Users\ASUS\Desktop\shoppin-assignment\resnet-model-trained")

['C:\\Users\\ASUS\\Desktop\\shoppin-assignment\\resnet-model-trained\\preprocessor_config.json']

### Load model checkpoint

In [ ]:
# Initialize the model first
model = CustomResNet(num_classes=10)  # or however many labels you have
# Load the saved state dictionary
model.load_state_dict(torch.load(r"C:\Users\ASUS\Desktop\shoppin-assignment\resnet-model-trained\model.pth"))
model.eval()  # Set to evaluation mode

c:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15912\2217045441.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.

CustomResNet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [ ]:
from tqdm import tqdm

def modified_forward(self, pixel_values):
    # Get features from the layer before the final classification layer
    # Remove the final classification layer temporarily
    original_fc = self.resnet.fc
    self.resnet.fc = nn.Identity()
    
    # Get embeddings
    with torch.no_grad():
        embeddings = self.resnet(pixel_values)
    
    # Restore the original classification layer
    self.resnet.fc = original_fc
    return embeddings

# Replace model's forward method temporarily
model.forward = modified_forward.__get__(model)

# Run inference to get the embeddings
embeddings = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for batch in tqdm(preprocessed_test_ds, desc="Extracting embeddings"):
    try:
        # Get the preprocessed pixel values
        inputs = batch['pixel_values']
        # Convert to tensor and ensure correct shape
        inputs = torch.tensor(inputs).float().to(device)  # Convert to float tensor
        if inputs.dim() == 3:
            inputs = inputs.unsqueeze(0)  # Add batch dimension if needed
        
        with torch.no_grad():
            embedding = model(pixel_values=inputs)
            # Move to CPU and convert to numpy
            embedding = embedding.cpu().numpy()
            embeddings.append(embedding)
    except Exception as e:
        print(f"Error processing batch: {str(e)}")
        continue

# Concatenate all embeddings
embeddings = np.concatenate(embeddings, axis=0)

# Create vectors dictionary
vectors = {test_ids[i]: embeddings[i] for i in range(len(test_ids))}

# Verify the embeddings
print(f"\nNumber of embeddings: {len(vectors)}")
print(f"Embedding dimension: {embeddings.shape[1] if len(embeddings.shape) > 1 else 'Unknown'}")

# Optional: Print a sample embedding to verify the values
sample_id = next(iter(vectors.keys()))
print(f"\nSample embedding for ID {sample_id}:")
print(f"Shape: {vectors[sample_id].shape}")
print(f"First few values: {vectors[sample_id][:5]}")








































































































































































































































































Extracting embeddings: 100%|██████████| 524/524 [00:32<00:00, 16.26it/s]


Number of embeddings: 524
Embedding dimension: 2048

Sample embedding for ID 15970:
Shape: (2048,)
First few values: [0.1663723  0.33546552 0.03094349 0.9092974  1.0120952 ]


In [ ]:
len(vectors.keys())

524

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def compute_similarity_matrix(embeddings):
    """
    Compute a pairwise cosine similarity matrix from the embeddings.
    """
    embeddings = np.array(embeddings)
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix

def evaluate_similarity(vectors, labels, top_k=5):
    """
    Evaluate the model using embeddings for image similarity.
    
    Args:
        vectors (dict): A dictionary of image IDs and their corresponding embeddings.
        labels (list): Ground truth labels for the images.
        top_k (int): Number of top results to consider for evaluation.

    Returns:
        dict: Dictionary of evaluation metrics.
    """
    # Convert vectors to matrix and maintain IDs
    ids = list(vectors.keys())  # Convert keys to list to ensure consistent ordering
    embeddings = [vectors[key] for key in ids]  # Use ids list directly
    
    # Convert embeddings to 2D array if needed
    embeddings = np.array(embeddings)
    if len(embeddings.shape) == 3:
        embeddings = embeddings.squeeze(1)  # Remove extra dimension if present
        
    similarity_matrix = compute_similarity_matrix(embeddings)

    # Evaluation metrics
    correct_top_k = 0
    total_queries = len(labels)
    
    # Convert labels to numpy array for easier indexing
    labels = np.array(labels)
    
    for idx in tqdm(range(len(ids)), desc="Evaluating Precision@K"):
        query_label = labels[idx]
        
        # Get similarities for current query
        similarities = similarity_matrix[idx].copy()  # Make a copy to avoid modifying original
        similarities[idx] = -np.inf  # Exclude self-match
        
        # Get top-k most similar image indices
        top_k_indices = np.argsort(similarities)[-top_k:]
        
        # Check if any of the top-k have the same label
        top_k_labels = labels[top_k_indices]
        if query_label in top_k_labels:
            correct_top_k += 1

    precision_at_k = correct_top_k / total_queries

    return {
        "precision@K": precision_at_k,
        "total_queries": total_queries,
        "correct_top_k": correct_top_k
    }


In [ ]:
# Run evaluation
results = evaluate_similarity(vectors, outputs.label_ids, top_k=1)
print("Topk=1 Evaluation Results:", results)

print('---------------------------------------------------------')

results = evaluate_similarity(vectors, outputs.label_ids, top_k=5)
print("Topk=5 Evaluation Results:", results)

print('---------------------------------------------------------')

results = evaluate_similarity(vectors, outputs.label_ids, top_k=10)
print("Topk=10 Evaluation Results:", results)


Evaluating Precision@K: 100%|██████████| 524/524 [00:00<00:00, 42307.99it/s]


Topk=1 Evaluation Results: {'precision@K': 0.851145038167939, 'total_queries': 524, 'correct_top_k': 446}
---------------------------------------------------------



Evaluating Precision@K: 100%|██████████| 524/524 [00:00<00:00, 53711.36it/s]


Topk=5 Evaluation Results: {'precision@K': 0.9713740458015268, 'total_queries': 524, 'correct_top_k': 509}
---------------------------------------------------------



Evaluating Precision@K: 100%|██████████| 524/524 [00:00<00:00, 36163.74it/s]

Topk=10 Evaluation Results: {'precision@K': 0.9866412213740458, 'total_queries': 524, 'correct_top_k': 517}
